# TSA Chapter 5: Rolling vs Multi-Step Forecast

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch5/TSA_ch5_rolling_forecast/TSA_ch5_rolling_forecast.ipynb)

Comparison of rolling one-step vs multi-step GARCH forecasts on Bitcoin.

In [ ]:
!pip install numpy pandas matplotlib scipy yfinance arch statsmodels -q

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy import stats
# Chart style
BLUE = '#1A3A6E'; RED = '#DC3545'; GREEN = '#2E7D32'
ORANGE = '#E67E22'; GRAY = '#666666'; PURPLE = '#8E44AD'

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'savefig.transparent': True,
    'axes.grid': False, 'font.size': 10, 'axes.labelsize': 11,
    'axes.titlesize': 12, 'legend.fontsize': 9, 'xtick.labelsize': 9,
    'ytick.labelsize': 9, 'axes.spines.top': False, 'axes.spines.right': False,
    'lines.linewidth': 1.0, 'axes.linewidth': 0.6,
    'legend.facecolor': 'none', 'legend.framealpha': 0, 'legend.edgecolor': 'none',
    'figure.dpi': 150,
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

def legend_bottom(ax, ncol=2, y=-0.18):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, y), ncol=ncol, frameon=False)
import yfinance as yf
from arch import arch_model

In [ ]:
import yfinance as yf

# Download Bitcoin data
btc = yf.download('BTC-USD', start='2019-01-01', end='2025-12-31', progress=False)
if isinstance(btc.columns, pd.MultiIndex):
    btc.columns = btc.columns.get_level_values(0)
btc['Return'] = btc['Close'].pct_change() * 100
btc.dropna(inplace=True)
btc_returns = btc['Return']
print(f"Bitcoin: {len(btc)} observations")

In [ ]:
# Rolling vs multi-step forecast
returns = btc_returns; n = len(returns); n_test = 120; n_train = n - n_test

rolling_vol = []
for i in range(n_test):
    window = returns.iloc[:n_train + i]
    mod = arch_model(window, vol='Garch', p=1, q=1, mean='Constant', dist='normal')
    r = mod.fit(disp='off', update_freq=0)
    fcast = r.forecast(horizon=1)
    rolling_vol.append(np.sqrt(fcast.variance.values[-1, 0]))
    if (i+1) % 30 == 0: print(f"  {i+1}/{n_test}")

window_ms = returns.iloc[:n_train]
mod_ms = arch_model(window_ms, vol='Garch', p=1, q=1, mean='Constant', dist='normal')
r_ms = mod_ms.fit(disp='off')
fcast_ms = r_ms.forecast(horizon=n_test)
multistep_vol = np.sqrt(fcast_ms.variance.values[-1, :])

test_idx = returns.index[n_train:n_train + n_test]
realized = np.abs(returns.iloc[n_train:n_train + n_test].values)
realized_smooth = pd.Series(realized, index=test_idx).rolling(10).mean()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharey=True)
axes[0].plot(test_idx, rolling_vol, color=BLUE, lw=0.9, label='Rolling 1-step')
axes[0].plot(test_idx, realized_smooth, color=GRAY, lw=0.7, alpha=0.6, label='$|r_t|$ (10d MA)')
axes[0].set_title('(a) Rolling One-Step Forecast', fontweight='bold')
axes[0].set_ylabel('Volatility (%)')
axes[0].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.setp(axes[0].get_xticklabels(), rotation=30, ha='right')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.25), ncol=2, frameon=False)

axes[1].plot(test_idx, multistep_vol, color=RED, lw=0.9, label='Multi-step')
axes[1].plot(test_idx, realized_smooth, color=GRAY, lw=0.7, alpha=0.6, label='$|r_t|$ (10d MA)')
axes[1].set_title('(b) Multi-Step Forecast', fontweight='bold')
axes[1].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.setp(axes[1].get_xticklabels(), rotation=30, ha='right')
axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.25), ncol=2, frameon=False)

fig.suptitle('Rolling vs Multi-Step GARCH Forecasts', y=1.02, fontsize=12)
fig.tight_layout(); fig.subplots_adjust(bottom=0.25)
save_chart(fig, 'garch_rolling_forecast'); plt.show()